In [6]:
import numpy as np
import tensorflow as tf
import keras
from keras.utils import to_categorical
import SimpleITK as sitk
import os
import matplotlib.pyplot as plt
import pydicom 
import cv2
print("Tensorflow version {} GPU {}".format(tf.__version__, tf.test.is_gpu_available()))
def read_DICOM(dir_path, slices_cutoff=0):
    'Read DICOM image from directory'
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames( dir_path )   
    if len(dicom_names) > slices_cutoff:
        reader.SetFileNames(dicom_names)
        try:
            image = reader.Execute()
        except:
            print("Error reading {}".format(dir_path))
            return None
        return image
    else:
        return None

def get_slice(im, axis=1, fract=0.5, flip=None):
    'Show slice along axis at fraction of slice'
    if isinstance(im, sitk.SimpleITK.Image):
        im = sitk.GetArrayFromImage(im)
    if flip != None:
        im = np.flip(im, flip)
    idx = int(im.shape[axis]*fract)
    if axis == 0:
        im_slice = im[idx, :, :]
    elif axis == 1:
        im_slice = im[:, idx, :]
    elif axis == 2:
        im_slice = im[:, :, idx]
    else:
        im_slice = im[idx, :, :]
    return im_slice

Tensorflow version 1.13.1 GPU True


In [11]:
images_dir_path = ('/home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/LIDC-IDRI')
# images_dir_path = ('/home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/LIDC-IDRI/LIDC-IDRI-0032')
slices_dir = '/home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images'
# slices_dir = '/home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images_32'
print(images_dir_path)
if not os.path.exists(slices_dir):
    os.makedirs(slices_dir)

TEST_FOLDER = "test"
TRAIN_FOLDER = "train"
VALIDATION_FOLDER = "validation"
data_splits = [TRAIN_FOLDER, TEST_FOLDER, VALIDATION_FOLDER]

test_dir = os.path.join(slices_dir, TEST_FOLDER)
train_dir = os.path.join(slices_dir, TRAIN_FOLDER)
validation_dir = os.path.join(slices_dir, VALIDATION_FOLDER)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)
    

/home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/LIDC-IDRI


In [12]:
import random
print(random.uniform(0,1))
XRAY = "xray"
axis_folders = ["0", "1", "2", XRAY]
fract = 0.5
axises = [0, 1, 2]

for axis in axis_folders:
    for split_folder in data_splits:
        axis_dir = os.path.join(slices_dir, split_folder, str(axis))
        if not os.path.exists(axis_dir):
            os.makedirs(axis_dir)

0.641137591226887


In [13]:
TEST_PROB = 0.2
VALIDATION_PROB = 0.1 
test_prob = TEST_PROB
validation_prob = VALIDATION_PROB

for root, subFolders, files in os.walk(images_dir_path):
    files = [f for f in files if f.endswith("dcm")]
    split_prob = random.uniform(0, 1)
    split_dir = None
    if split_prob < test_prob:
        split_dir = os.path.join(slices_dir, TEST_FOLDER)
    elif split_prob < test_prob + validation_prob:
        split_dir = os.path.join(slices_dir, VALIDATION_FOLDER)
    else:
        split_dir = os.path.join(slices_dir, TRAIN_FOLDER)
        
    if len(files) > 0:
        one_path = os.path.join(root, files[0])
        one_dcm = pydicom.dcmread(one_path)
        if len(files) > 0:
            image = read_DICOM(dir_path=root, slices_cutoff=2)
        if image and one_dcm:
            for axis in axises:
                axis_dir = os.path.join(split_dir, str(axis))
                im_slice = get_slice(image, axis=axis, fract=fract, flip=0)
                filename = "{}/{}.{}.slice.png".format(axis_dir, one_dcm.PatientID, axis)
                try:
                    cv2.imwrite(filename, im_slice)
                except:
                    print("Error writing {}".format(filename))
        elif not image and one_dcm:
            filename = "{}/{}/{}.{}.png".format(split_dir, XRAY, one_dcm.PatientID, XRAY)
            try:
                cv2.imwrite(filename, one_dcm.pixel_array)
            except:
                print("Error writing {}".format(filename))

Error reading /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/LIDC-IDRI/LIDC-IDRI-0081/01-01-2000-67696/3000987-91956
Error reading /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/LIDC-IDRI/LIDC-IDRI-0112/01-01-2000-30946/3001599-75345
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-1012.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-1012.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-1012.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-1012.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-1012.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/validation/xray/LID

Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/validation/xray/LIDC-IDRI-0963.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-0963.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/test/xray/LIDC-IDRI-0963.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/validation/xray/LIDC-IDRI-0963.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-0963.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/test/xray/LIDC-IDRI-0963.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/train/xray/LIDC-IDRI-0766.xray.png
Error writing /home/ktdiedrich/Documents/localData/RadiologyImages/LIDC-IDRI/slice_images/validation/xray/LIDC-IDRI-0